## Name : Vedant Mehta
## Class: BE Computers
## UID  : 2018130028

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.dates as mdates
import numpy as np

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Reading Data

In [3]:
sales = pd.read_csv("/kaggle/input/sales-forecasting/train.csv")

In [4]:
sales.head()

### We can see the column list
1. sales over time
2. category vs sales
3. sub-category vs sales
4. state vs sales
5. city vs sales
6. segment vs sales
7. top customers
8. count of shipmodes
9. region vs sales
10. leadtime histogram

In [5]:
sales.describe()

In [6]:
sales.info()

In [7]:
# replacing the spaces in the column names to underscore
sales.columns = sales.columns.str.replace(' ','_')

### Converting the dates from string to date format

In [8]:
sales["Order_Date"] = pd.to_datetime(sales["Order_Date"], dayfirst = True).dt.date
sales["Ship_Date"] = pd.to_datetime(sales["Ship_Date"], dayfirst = True).dt.date

### Creating a new column to show leadtime, which is the difference between the "ship_date" and "order_date"

In [9]:
sales["Leadtime"] = sales["Ship_Date"] - sales["Order_Date"]

In [10]:
sales["Month"] = pd.DatetimeIndex(sales["Order_Date"]).month
sales["Year"] = pd.DatetimeIndex(sales["Order_Date"]).year

### Sum group by order_date

In [11]:
sales.groupby("Year")["Sales"].sum()

In [12]:
import matplotlib.pyplot as plt

In [13]:
sales.groupby("Year")["Sales"].sum().plot()
plt.ylabel("Sales")
plt.title("Trend of Total sales per year")

### Average sales group by month

In [14]:
sales.groupby("Month")["Sales"].mean()

In [15]:
sales.groupby("Month")["Sales"].mean().plot.bar()
plt.ylabel("Sales")
plt.title("Average Sales for each month")

### Average sales grouped by City

In [18]:
sales.groupby("City")["Sales"].sum().sort_values(ascending=False).head(10)

In [19]:
plt.figure(figsize=(10,5))
sales.groupby("City")["Sales"].sum().sort_values(ascending=False).head(10).plot.bar()
plt.ylabel("Sales")
plt.title("Top 10 Cities in terms of Sales")

### Average sales grouped by category

In [20]:
sales.groupby("Category")["Sales"].sum().sort_values(ascending=False)

In [21]:
sales.groupby("Category")["Sales"].sum().sort_values().plot.barh()
plt.ylabel("Sales")
plt.title("Total Sales per category")

In [22]:
plt.figure(figsize=(10,10))
sales.groupby("Sub-Category")["Sales"].sum().sort_values().plot.barh()
plt.ylabel("Sales")
plt.title("Total Sales per Sub-category")

### Average sales grouped by state then city

In [23]:
sales.groupby(["State","City"])["Sales"].sum().sort_values(ascending=False)

### Average sales grouped by segment

In [24]:
sales.groupby("Segment")["Sales"].sum().sort_values(ascending=False)

In [25]:
sales.groupby("Segment")["Sales"].sum().sort_values().plot.barh()
plt.ylabel("Sales")
plt.title("Total Sales per Segment")

In [30]:
sales.head()

In [31]:
sales.info()

In [32]:
sales["Leadtime"].dt.days.describe()

In [36]:
sales["Year_Month"] = pd.to_datetime(sales["Year_Month"]).dt.date

In [38]:
# sales trend using year_month
plt.figure(figsize=(20,5))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=75))
sales.groupby("Year_Month")["Sales"].sum().plot()
plt.ylabel("Sales")
plt.title("Sales Trend by Year-Month")

In [39]:
# sales trend using year_month
plt.figure(figsize=(20,5))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=75))
sales[sales.Category == "Furniture"].groupby("Year_Month")["Sales"].sum().plot(label="Furniture")
sales[sales.Category == "Office Supplies"].groupby("Year_Month")["Sales"].sum().plot(label="Office Supplies")
sales[sales.Category == "Technology"].groupby("Year_Month")["Sales"].sum().plot(label="Technology")
plt.ylabel("Sales")
plt.legend()
plt.title("Sales Trends of each Category")

In [40]:
sales_trend = sales[["Year_Month","Sales"]].groupby("Year_Month").sum()
sales_trend = sales_trend.sort_values(by="Year_Month")
sales_trend.head()

In [41]:
# compute for rolling average
sales_trend["rolmean"] = sales_trend.rolling(window=5).mean()
sales_trend.head()

In [42]:
# sales trend using year_month with rolling average
sales_trend.plot(figsize=(20,5))
plt.legend()
plt.title("Sales Trend by Year-Month with Rolling Average")

In [43]:
# create a new dataframw where sales are aggregated monthly, then show date & monthly total sales only
sales_data = sales[["Year_Month","Sales"]].groupby("Year_Month").sum()
sales_data = sales_data.sort_values(by="Year_Month")
sales_data.head()

In [44]:
# use adfuller to check if sales trend is stationary
from statsmodels.tsa.stattools import adfuller
result = adfuller(sales_data)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

In [51]:
# use ARIMA for forecasting
from statsmodels.tsa.arima.model import ARIMA
model=ARIMA(sales_shift,order=(1,1,1))
model_fit=model.fit()
model_fit.summary()

In [52]:
# plot forecast
sales_data['forecast']=model_fit.predict(start=25,end=50,dynamic=True)
sales_data[['Sales','forecast']].plot(figsize=(12,8))

In [53]:
# use sarimax instead due to seasonality
import statsmodels.api as sm
model=sm.tsa.statespace.SARIMAX(sales_data['Sales'],order=(1, 1, 0),seasonal_order=(1,1,0,12))
results=model.fit()
sales_data['forecast']=results.predict(start=35,end=50,dynamic=True)
sales_data[['Sales','forecast']].plot(figsize=(12,8))

In [54]:
# extend forecast until 2022
from pandas.tseries.offsets import DateOffset
future_dates=[sales_data.index[-1]+ DateOffset(months=x)for x in range(0,36)]
future_datest_df=pd.DataFrame(index=future_dates[1:],columns=sales_data.columns)

future_datest_df.tail()

future_df=pd.concat([sales_data,future_datest_df])

future_df['forecast'] = results.predict(start = 43, end = 200, dynamic= True)
future_df[['Sales', 'forecast']].plot(figsize=(12, 8))